In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

import math

In [4]:
data=pd.read_csv('btcusdt_30m.csv')
data

,datetime,open,high,low,close,volume
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13521.12,221.752443
1,2018-01-01 06:00:00,13500.00,13690.87,13450.00,13529.01,221.603756
2,2018-01-01 06:30:00,13528.99,13595.89,13402.28,13560.00,160.399291
3,2018-01-01 07:00:00,13559.99,13559.99,13155.38,13203.06,223.297715
4,2018-01-01 07:30:00,13203.00,13411.76,13202.03,13225.00,203.728509
...,...,...,...,...,...,...
70394,2022-01-12 03:30:00,42664.71,42733.00,42597.41,42650.75,337.739550
70395,2022-01-12 04:00:00,42647.14,42776.14,42633.18,42713.13,224.120380
70396,2022-01-12 04:30:00,42713.12,42886.28,42665.26,42790.01,358.331130
70397,2022-01-12 05:00:00,42790.00,42822.21,42633.97,42729.29,322.810880


In [7]:
output = pd.read_csv('rsi_data.csv')
output

,Unnamed: 0,datetime,signal
0,0,2018-01-01 05:30:00,0
1,1,2018-01-01 06:00:00,0
2,2,2018-01-01 06:30:00,0
3,3,2018-01-01 07:00:00,0
4,4,2018-01-01 07:30:00,0
...,...,...,...
70394,70394,2022-01-12 03:30:00,0
70395,70395,2022-01-12 04:00:00,0
70396,70396,2022-01-12 04:30:00,0
70397,70397,2022-01-12 05:00:00,0


In [8]:
def to_submit(data,output):
    datetime=[]
    trade=[]
    signal=[]
    price=[]
    current=0
    for i in range(len(data.close)):
        if(i==(len(data.close)-1)):
            if(current==1):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(-1)
            elif(current==-1):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(1)
            else:
                break
            break
        if(output.signal[i]==1):
            if(current==1):
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(0)
            elif(current==0):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(1)
                current=1
            else:
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(1)
                current=0
            
        elif(output.signal[i]==-1):
            if(current==1):
                datetime.append(output.datetime[i])
                signal.append(-1)
#                 price.append(data.close[i])
                trade.append('close')
                current=0
            elif(current==0):
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(-1)
                current=-1
            else:
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(0)
                current=-1
        
        else:
            datetime.append(output.datetime[i])
            trade.append('close')
#             price.append(data.close[i])
            signal.append(0)
        
            
            
    to_submit=pd.DataFrame(columns=['datetime','signal','close\open'])
    to_submit['datetime']=datetime
    to_submit['signal']=signal
    to_submit['close\open']=trade
#     to_submit['price']=price
    return to_submit
            
            

In [9]:
def metrics(data,submit):
    capital=1000
    current=0
    temp=0
    portfolio_value=[]
    datetime=[]
    quantity=[]
    
    sharpe=[]
    returns=[]
    risk_free_rate=0.05       
    for i in range(len(data.close)):
        datetime.append(data.datetime[i])
        if(submit.signal[i]==1):
            if(current==0):
                portfolio_value.append(capital)
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
                
                temp=capital/(data.close[i])
                quantity.append(temp)
                capital=0
                current=1
            else:
                capital=capital-(temp*data.close[i])
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
                quantity.append(temp)
                portfolio_value.append(capital)
                temp=0
                current=0
                
        elif(submit.signal[i]==-1):
            if(current==0):
                portfolio_value.append(capital)
                temp=capital/(data.close[i])
                quantity.append(temp)
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
                capital=capital+(temp*data.close[i])
                
                current=-1
            else:
                capital=capital+(temp*data.close[i])
                quantity.append(temp)
                current=0
                temp=0
                portfolio_value.append(capital)
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
        
        else:
            if(current==0):
                portfolio_value.append(capital)
                quantity.append(temp)
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
            elif(current==1):
                portfolio_value.append(capital+(temp*data.close[i]))
                quantity.append(temp)
                returns.append((capital+(temp*data.close[i])-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
            else:
                portfolio_value.append(capital-(temp*data.close[i]))
                quantity.append(temp)
                returns.append((capital-(temp*data.close[i])-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
                
    a=pd.DataFrame(columns=['datetime','portfolio value'])
    a['datetime']=datetime
    a['portfolio value']=portfolio_value
#     a['sharpe']=sharpe
    a['quantity']=quantity
    
#     a['daily_return'] = a['portfolio value'].pct_change()
#     daily_volatility = a['daily_return'].std()
#     annual_volatility = daily_volatility * math.sqrt(252)
#     risk_free_rate = 0.03
    
#     a['excess_return'] = a['daily_return'] - risk_free_rate / 252
#     cumulative_excess_return = a['excess_return'].sum()
#     negative_returns = [r for r in a['excess_return'] if r < 0]
#     cumulative_volatility = a['daily_return'].std() * math.sqrt(252)
#     hello=np.std(negative_returns)*math.sqrt(252)
#     sharpe_ratio = cumulative_excess_return / cumulative_volatility
#     sortino_ratio=cumulative_excess_return / hello

    a=pd.DataFrame(columns=['datetime','portfolio value'])
    a['datetime']=datetime
    a['portfolio value']=portfolio_value
    a['quantity']=quantity
    a['daily_return'] = a['portfolio value'].pct_change()
    daily_volatility = a['daily_return'].std()
    annual_volatility = daily_volatility * math.sqrt(365)
    annual_volatility_negative = a[a['daily_return'] < 0]['daily_return'].std() * math.sqrt(365)
    risk_free_rate = 0.05
    a['excess_return'] = (a['daily_return'] - risk_free_rate)
    annual_excess_return = (a['daily_return'].mean()*365 - risk_free_rate)
    cumulative_excess_return = a['excess_return'].sum()
    negative_returns = [r for r in a['excess_return'] if r < 0]
    cumulative_volatility = a['daily_return'].std()*math.sqrt(252)
    hello=np.std(negative_returns)*math.sqrt(252)
    sharpe_ratio = annual_excess_return / annual_volatility
    sortino_ratio = annual_excess_return / annual_volatility_negative
    
    
    
    start=[]
    end=[]
    start_value=[]
    end_value=[]
    duration=[]
    P_and_L=[]
    return_for_every_trade=[]
    pointer=0
    drawdown=[]
    for i in range(len(submit.signal)):
        if(submit.signal[i]==1):
            if(pointer==0):
                start.append(i)
                start_value.append(portfolio_value[i])
                pointer=1
            else:
                end.append(i)
                end_value.append(portfolio_value[i])
                return_for_every_trade.append(100*((end_value[-1]/start_value[-1])-1))
                P_and_L.append(end_value[-1]-start_value[-1])
                duration.append(end[-1]-start[-1])
                pointer=0
                minimum=np.min(portfolio_value[start[-1]:end[-1]])
                maximum=np.max(portfolio_value[start[-1]:end[-1]])
                drawdown.append(100*((maximum-minimum)/maximum))
#                 drawdown.append(100*((portfolio_value[start[-1]]/minimum)-1))
                
        elif(submit.signal[i]==-1):
            if(pointer==0):
                start.append(i)
                start_value.append(portfolio_value[i])
                pointer=-1
            else:
                end.append(i)
                end_value.append(portfolio_value[i])
                return_for_every_trade.append(100*((end_value[-1]/start_value[-1])-1))
                P_and_L.append(end_value[-1]-start_value[-1])
                duration.append(end[-1]-start[-1])
                pointer=0
                minimum=np.min(portfolio_value[start[-1]:end[-1]])
                maximum=np.max(portfolio_value[start[-1]:end[-1]])
                drawdown.append(100*((maximum-minimum)/maximum))
                
    b=pd.DataFrame(columns=['start','end'])
    b['start']=start
    b['end']=end
    b['duration']=duration
    b['return for every trade']=return_for_every_trade
    b['P and L']=P_and_L
    b['drawdown']=drawdown
    
    

    gross_profit=[]
    gross_loss=[]
    for i in range(len(P_and_L)):
        if(P_and_L[i]>0):
            gross_profit.append(P_and_L[i])
        else:
            gross_loss.append(P_and_L[i])
    print("GROSS PROFIT: ",np.sum(gross_profit))
    print("GROSS LOSS: ",np.sum(gross_loss))
    print("NET P and L: ",np.sum(P_and_L))
    print("TOTAL CLOSED TRADES: ",len(start))
    print("WIN RATE: ",100*(len(gross_profit)/(len(gross_profit)+len(gross_loss))))
    print("AVG WINNING TRADE: ",np.mean(gross_profit))
    print("AVG LOSING TRADE: ",np.mean(gross_loss))
    print("Largest Losing trade: ",min(gross_loss))
    print("Largest winning trade: ",max(gross_profit))
    print("Avg holding duration: ",np.mean(duration))
    
   
    
    print("Sharpe Ratio: ",sharpe_ratio)


#     std_dev_negative = np.std(negative_returns)
            
    print("Sortino Ratio: ",sortino_ratio)
    print("RETURNS: ",((np.sum(P_and_L))/10))
    print("MAX Drawdown: (in %) ",np.max(drawdown))
    
    return a,b
                

In [10]:
daily_metrics,only_trade=metrics(data,to_submit(data,output))

C:\Users\Rachit\AppData\Local\Temp\ipykernel_23272\3408008015.py:57: RuntimeWarning: divide by zero encountered in double_scalars
  sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
C:\Users\Rachit\AppData\Local\Temp\ipykernel_23272\3408008015.py:39: RuntimeWarning: divide by zero encountered in double_scalars
  sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))


GROSS PROFIT:  1744.6266622833423
GROSS LOSS:  -2735.2435486379354
NET P and L:  -990.6168863545927
TOTAL CLOSED TRADES:  420
WIN RATE:  59.523809523809526
AVG WINNING TRADE:  6.978506649133369
AVG LOSING TRADE:  -16.089667933164325
Largest Losing trade:  -196.5780209306398
Largest winning trade:  101.81672817391996
Avg holding duration:  152.06190476190477
Sharpe Ratio:  -0.4934185424961545
Sortino Ratio:  -0.5911955467477985
RETURNS:  -99.06168863545926
MAX Drawdown: (in %)  71.78025281725601


In [11]:
daily_metrics

,datetime,portfolio value,quantity,daily_return,excess_return
0,2018-01-01 05:30:00,1000.000000,0.000000,NaN,NaN
1,2018-01-01 06:00:00,1000.000000,0.000000,0.000000,-0.050000
2,2018-01-01 06:30:00,1000.000000,0.000000,0.000000,-0.050000
3,2018-01-01 07:00:00,1000.000000,0.000000,0.000000,-0.050000
4,2018-01-01 07:30:00,1000.000000,0.000000,0.000000,-0.050000
...,...,...,...,...,...
70394,2022-01-12 03:30:00,9.340998,0.000219,-0.000198,-0.050198
70395,2022-01-12 04:00:00,9.354660,0.000219,0.001463,-0.048537
70396,2022-01-12 04:30:00,9.371497,0.000219,0.001800,-0.048200
70397,2022-01-12 05:00:00,9.358199,0.000219,-0.001419,-0.051419


In [12]:
only_trade

,start,end,duration,return for every trade,P and L,drawdown
0,84,318,234,-11.343006,-113.430063,20.338385
1,321,918,597,-21.311007,-188.936982,44.940837
2,919,971,52,6.091218,42.494347,9.249007
3,972,1085,113,-4.439140,-32.855288,17.173684
4,1086,1213,127,6.924153,48.972598,9.289968
...,...,...,...,...,...,...
415,69681,69839,158,-2.779543,-0.246034,6.854030
416,69840,69859,19,3.535512,0.304251,2.915123
417,69860,70043,183,2.707237,0.241210,4.318729
418,70044,70101,57,5.627492,0.514973,3.269749


In [13]:

def only_long_then_short(data,output):
    datetime=[]
    signal=[]
    trade=[]
    price=[]
    current=0
    for i in range(len(data.close)):
        datetime.append(data.datetime[i])
        trade.append('close')
#         price.append(data.close[i])
        if(i==len(data.close)-1):
            if(current==1):
                signal.append(-1)
                break
            else:
                signal.append(0)
                break
            
        
        if(output.signal[i]==1):
            if(current==0):
                signal.append(1)
                current=1
            else:
                signal.append(0)
        elif(output.signal[i]==-1):
            if(current==1):
                signal.append(-1)
                current=0
            else:
                signal.append(0)
            
        else:
            if(current==0):
                signal.append(0)
            else:
                signal.append(0)
    a=pd.DataFrame(columns=['datetime','signal','trade','price'])
    a['price']=price
    a['datetime']=datetime
    a['signal']=signal
    a['trade']=trade
    return a
            

In [14]:
only_long_then_short_daily,only_long_then_short_trade=metrics(data,only_long_then_short(data,output))

C:\Users\Rachit\AppData\Local\Temp\ipykernel_23272\3408008015.py:57: RuntimeWarning: divide by zero encountered in double_scalars
  sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
C:\Users\Rachit\AppData\Local\Temp\ipykernel_23272\3408008015.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))


GROSS PROFIT:  2024.8372788348202
GROSS LOSS:  -2732.83588618006
NET P and L:  -707.9986073452394
TOTAL CLOSED TRADES:  233
WIN RATE:  61.80257510729614
AVG WINNING TRADE:  14.061369991908473
AVG LOSING TRADE:  -30.70602119303438
Largest Losing trade:  -228.7495179684488
Largest winning trade:  90.08654806780328
Avg holding duration:  153.04721030042919
Sharpe Ratio:  -0.5399275856125357
Sortino Ratio:  -0.4650224117287019
RETURNS:  -70.79986073452395
MAX Drawdown: (in %)  48.031805143173465


In [15]:
only_long_then_short_daily,

(                  datetime  portfolio value  quantity  daily_return  \
 0      2018-01-01 05:30:00      1000.000000  0.000000           NaN   
 1      2018-01-01 06:00:00      1000.000000  0.000000      0.000000   
 2      2018-01-01 06:30:00      1000.000000  0.000000      0.000000   
 3      2018-01-01 07:00:00      1000.000000  0.000000      0.000000   
 4      2018-01-01 07:30:00      1000.000000  0.000000      0.000000   
 ...                    ...              ...       ...           ...   
 70394  2022-01-12 03:30:00       290.690751  0.006816     -0.000198   
 70395  2022-01-12 04:00:00       291.115909  0.006816      0.001463   
 70396  2022-01-12 04:30:00       291.639893  0.006816      0.001800   
 70397  2022-01-12 05:00:00       291.226049  0.006816     -0.001419   
 70398  2022-01-12 05:30:00       292.001393  0.006816      0.002662   
 
        excess_return  
 0                NaN  
 1          -0.050000  
 2          -0.050000  
 3          -0.050000  
 4          -0

In [16]:
only_long_then_short_trade

,start,end,duration,return for every trade,P and L,drawdown
0,318,918,600,-22.874952,-228.749518,44.940837
1,971,1085,114,-4.315253,-33.281407,17.173684
2,1213,1302,89,12.207361,90.086548,10.689022
3,1392,1609,217,-14.710811,-121.813696,27.580223
4,1653,2066,413,8.581851,60.608627,33.115439
...,...,...,...,...,...,...
228,69164,69252,88,1.723268,5.155220,3.603568
229,69589,69625,36,2.211550,6.729944,1.401737
230,69680,69839,159,-3.420779,-10.639950,7.450877
231,69859,70043,184,1.385778,4.162862,4.318729
